In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import string
import spacy
import jaro
import re

In [14]:
def get_sheet(src):
    raw_html = requests.get(src).text
    soup = BeautifulSoup(raw_html, 'html.parser')
    table = soup.find("tbody")
    rows = table.find_all("tr")
    grid = []
    for row in rows:
        cols = row.find_all("td")
        clean_row = []
        for col in cols:
            clean_row.append(col.text)
        grid.append(clean_row)
    return grid

get_sheet("https://docs.google.com/spreadsheets/d/1SdgUPuYbxCl-AK9bLmHiizCrYsIXEQQz5PpBPCqO-rg/edit?usp=sharing")

[['work_year',
  'experience_level',
  'employment_type',
  'job_title',
  'salary',
  'salary_currency',
  'salary_in_usd',
  'employee_residence',
  'remote_ratio',
  'company_location',
  'company_size',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['2023',
  'SE',
  'FT',
  'Principal Data Scientist',
  '80000',
  'EUR',
  '85847',
  'ES',
  '100',
  'ES',
  'L',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['2023',
  'MI',
  'CT',
  'ML Engineer',
  '30000',
  'USD',
  '30000',
  'US',
  '100',
  'US',
  'S',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['2023',
  'MI',
  'CT',
  'ML Engineer',
  '25500',
  'USD',
  '25500',
  'US',
  '100',
  'US',
  'S',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['2023',
  'SE',
  'FT',
  'Data Scientist',
  '175000',
  'USD',
  '175000',
  'CA',
  '100',
  

In [15]:
def sheet_to_df(src):
    sheet = get_sheet(src)

    #A dataframe starts when a header is found
    #A header is a the first instance of a set of contiguous columns in the same row that contain at least some letters
    #A dataframe ends when a blank row is found or an empty column is found

    num = 0 #The number of the dataframe
    headers = [] #Each header is a tuple (num, row, col_start, col_end)
    binding_headers = []
    dfs = [] #Each df is a tuple (num, df)

    #First pass: get all the headers
    for row in range(len(sheet)):

        #if every cell in the row is empty, skip row
        if all([sheet[row][col].strip() == "" for col in range(len(sheet[row]))]):
            headers += binding_headers
            binding_headers = []
            continue

        for col in range(len(sheet[row])):

            #Check if the cell is bounded by a header
            if any([col >= header[2] and col <= header[3] for header in binding_headers]):
                continue

            #Check if the cell is commented out
            if sheet[row][col].strip().startswith("//"):
                continue
            
            if re.search('[a-zA-Z]', sheet[row][col]):
                head_start = col
                head_end = col
                while head_end < len(sheet[row]) and re.search('[a-zA-Z]', sheet[row][head_end]):
                    head_end += 1
                    print(sheet[row][head_end])
                binding_headers.append([num, row, head_start, head_end])
                num += 1   
    headers += binding_headers

    #Second pass: get all the dataframes
    for header in headers:
        df = []
        for row in range(header[1], len(sheet)):
            print(header[2], header[3])
            if all([sheet[row][col].strip() == "" for col in range(header[2], header[3])]):
                break
            df_row = []
            for col in range(header[2], header[3]):
                df_row.append(sheet[row][col])
            df.append(df_row)
        cols = df[0]
        data = df[1:]
        df = pd.DataFrame(data, columns=cols)
        dfs.append(df)
    
    return dfs

# dfs = sheet_to_df("https://docs.google.com/spreadsheets/d/1SJngKcNkzHupfVBSLLAwyZ5bzHILNRggxsjfXyLMI98/edit?usp=sharing")
dfs = sheet_to_df("https://docs.google.com/spreadsheets/d/1SdgUPuYbxCl-AK9bLmHiizCrYsIXEQQz5PpBPCqO-rg/edit?usp=sharing")
dfs

experience_level
employment_type
job_title
salary
salary_currency
salary_in_usd
employee_residence
remote_ratio
company_location
company_size

0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11
0 11


[   work_year experience_level employment_type                  job_title  \
 0       2023               SE              FT   Principal Data Scientist   
 1       2023               MI              CT                ML Engineer   
 2       2023               MI              CT                ML Engineer   
 3       2023               SE              FT             Data Scientist   
 4       2023               SE              FT             Data Scientist   
 ..       ...              ...             ...                        ...   
 94      2023               SE              FT             Data Scientist   
 95      2023               EN              FT  Machine Learning Engineer   
 96      2023               EN              FT  Machine Learning Engineer   
 97      2023               SE              FT              Data Engineer   
 98      2023               SE              FT              Data Engineer   
 
     salary salary_currency salary_in_usd employee_residence remote_ratio 

In [16]:
def select_df(prompt, dfs):

    #Simplify the prompt to just the nouns
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(prompt)
    nouns = [token.text for token in doc if (token.pos_ == 'NOUN' or token.pos_ == 'PROPN')]
    nouns = [word.lower() for word in nouns] #lowercase all the words (done after finding the nouns to preserve proper nouns)

    all_columns = [df.columns for df in dfs]

    #Find the similarity between the prompt subject and each df
    similarities = []
    for columns in all_columns:
        #Preprocess the columns
        cleaned_columns = [col.lower().strip().translate(str.maketrans('', '', string.punctuation)).split() for col in columns]
        cleaned_columns = [word for col in cleaned_columns for word in col] #flatten the list

        #Find the similarity
        similarity = 0
        for col in cleaned_columns:
            for word in nouns:
                similarity = max(jaro.jaro_winkler_metric(col, word), similarity)

        similarities.append(similarity)

    #Return the df with the highest similarity
    return dfs[similarities.index(max(similarities))]

prompt = "What is the median number of A"
df = select_df(prompt, dfs)
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
94,2023,SE,FT,Data Scientist,70000,USD,70000,US,0,US,M
95,2023,EN,FT,Machine Learning Engineer,163196,USD,163196,US,0,US,M
96,2023,EN,FT,Machine Learning Engineer,145885,USD,145885,US,0,US,M
97,2023,SE,FT,Data Engineer,217000,USD,217000,US,100,US,M
